In [1]:
import pandas as pd
import numpy as np

import requests
import json
import seaborn as sns
from bs4 import BeautifulSoup
import random
#import matplotlib.pyplot as plt
from collections import Counter
import ast
#import sqlite3
import re
from urllib3.exceptions import MaxRetryError
#import threading
#import time
#import timeit
import html2text
#import traceback
#import plotly.graph_objects as go
#import tldextract
from urllib.parse import urlparse
import ssl
import importlib


from tqdm.auto import tqdm
tqdm.pandas()

#from urllib import parse
#from langdetect import detect
#from requests.exceptions import ReadTimeout, TooManyRedirects, ConnectionError, ConnectTimeout,\
#    InvalidSchema, InvalidURL
#from langdetect.lang_detect_exception import LangDetectException
#from urllib3.exceptions import ProtocolError
#import urllib.request
#import socket
#import langid
#from requests.packages.urllib3.exceptions import InsecureRequestWarning
#requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

#from qwikidata.linked_data_interface import get_entity_dict_from_api
#from qwikidata.sparql import return_sparql_query_results

import mechanicalsoup
from boilerpy3 import extractors

In [2]:
reference_html_df = pd.read_csv('text_reference_sampled_df_html.csv')

# Get HTML to Extracted Raw Sentences

In [3]:
reference_html_df.head()

,reference_id,reference_property_id,reference_datatype,url,netloc,netloc_agg,error_msg,code,content-type,final_url,reason,language_crawl,language_crawl_score,sampling_weight_vb,sampling_weight,html
0,390d6c6e68a32e11f8d7b0883cda0557db529fe6,P4656,url,https://en.wikipedia.org/w/index.php?title=G._...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=G._...,OK,en,0.934613,321744|13,24749.538462,"<html class=""client-js ve-available"" lang=""en""..."
1,49b9aec8e10815611ff0379a34d5fd7c3830566e,P4656,url,https://en.wikipedia.org/w/index.php?title=Seb...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Seb...,OK,en,0.646651,321744|13,24749.538462,"<html class=""client-js ve-available"" lang=""en""..."
2,ab3e9ada7246257ffbfb86fa90a54f25e45a704e,P4656,url,https://en.wikipedia.org/w/index.php?title=Ave...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Ave...,OK,en,0.842498,321744|13,24749.538462,"<html class=""client-js ve-available"" lang=""en""..."
3,76b04346ad57869d9e5ae1007ba8343d708ab6f9,P4656,url,https://en.wikipedia.org/w/index.php?title=Yut...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Yut...,OK,en,0.608352,321744|13,24749.538462,"<html class=""client-js ve-available"" lang=""en""..."
4,1ee71b39caf6df395c64b436fea4895692812d38,P4656,url,https://en.wikipedia.org/w/index.php?title=Dol...,en.wikipedia.org,en.wikipedia.org,none,200,text/html; charset=UTF-8,https://en.wikipedia.org/w/index.php?title=Dol...,OK,en,0.856581,321744|13,24749.538462,"<html class=""client-js ve-available"" lang=""en""..."


In [4]:
import re
import string
_RE_COMBINE_WHITESPACE = re.compile(r"\s+")

def get_url_language(html):
    try:
        soup = BeautifulSoup(html, "html5")
        [s.decompose() for s in soup("script")]  # remove <script> elements
        if soup.body == None:
            return ('no body', None)
        body_text = _RE_COMBINE_WHITESPACE.sub(" ", soup.body.get_text(' ')).strip()
        return predict_language(body_text, k=1)[0]
    except Exception:
        raise
        
def get_text_p_tags(soup):
    p_tags = soup.find_all('p')
    text = [p.getText().strip() for p in p_tags if p.getText()]
    return '\n'.join(text)

def clean_text_line_by_line(text, join=True, ch_join = ' ', verb=True):
    # text = soup.body.get_text()
    # break into lines and remove leading and trailing space on each
    lines = list(text.splitlines())
    lines = (line.strip() for line in lines)
    # for each line, lets correct double spaces into single space
    lines = (re.sub(r' {2,}', ' ', line) for line in lines)
    # for each line, lets correct punctuation spaced to the left    
    lines = (re.sub(r' ([.,:;!?\\-])', r'\1', line) for line in lines)       
    # put periods if missing    
    lines = [line+'.' if line and line[-1] not in string.punctuation else line for i, line in enumerate(lines)]  
    
    if verb:
        for i, line in enumerate(lines):
            print(i,line)
    # drop blank lines
    if join:
        return ch_join.join([line for line in lines if line])
    else:
        return [line for line in lines if line]

In [5]:
def apply_manual_rules(text):
    # RULE: A line ending with a ':' followed by whitespaces and a newline is likely a continuing line and should be joined
    #text = re.sub(
    #   r':\s*\n', 
    #   r': ', 
    #   text
    #)
    # RULE: Remove [1] reference numbers
    text = re.sub(r'\[[0-9]+\]', '', text)
    return text

In [6]:
import justext

# USING DIFFERENT EXTRACTORS MIGHT BE A GOOD THING FOR THE PAPER
#extractor = extractors.ArticleExtractor()# DOES NOT EXTRACT SMALL TEXT AREAS
#extractor = extractors.DefaultExtractor()# FAILED AT FINDAGRAVE
#extractor = extractors.CanolaExtractor() # MISSES SOME SMALL SENTENCES IN PARAGRAPHS
#extractor = extractors.ArticleSentencesExtractor() # DOES NOT EXTRACT ALL SENTENCES, FAILED AT WIKIPEDIA
#extractor = extractors.NumWordsRulesExtractor() # MISSES SOME SMALL SENTENCES IN PARAGRAPHS
#extractor = extractors.LargestContentExtractor()# DOES NOT EXTRACT ALL SENTENCES, FAILED AT WIKIPEDIA

import re
_RE_COMBINE_WHITESPACE = re.compile(r"\s+")

def retrieve_text_from_html(html, soup_parser = 'html5lib', verb=True, join=True):
    soup = BeautifulSoup(html, soup_parser)
    for script in soup(["script", "style"]):
        script.decompose()
    if soup.body == None:
        return 'No body'
    [s.unwrap() for s in soup.body.find_all('strong')]
    
    # HTML parsing rules
    
    #TEXT INSIDE A PARAGRAPH TAG SHOULD NOT BE SEPARETED BY WHITESPACES OTHER THAN NORMAL SPACES
    for p in soup.body.find_all('p'):
        p.string = _RE_COMBINE_WHITESPACE.sub(" ", p.get_text('')).strip()
    
    #DECOMPOSE ALL BAD TAGS
    #--------------
    #for c in ['warningbox', 'metadata', 'references', 'navbox', 'toc', 'catlinks']:
    #    for e in soup.body.find_all(class_=c):
    #        print('decomposed',e)
    #        e.decompose()
    
    # DECOMPOSE INVISIBLE ELEMENTS
    #for e in soup.body.find_all():
    #    if e.hidden:
    #        print('decomposed',e)
    #        e.decompose()
    #    else:
    #        if e.attrs is not None:
    #            #print(e)
    #            #print('-')
    #            style = e.get('style')
    #            if style and 'display' in style and 'none' in style:
    #                print('decomposed',e)
    #                e.decompose()
    #                #print(e, style)
    #--------------
    
    #print(soup.body)
    
    # BOILERPLATE REMOVAL OPTIONS
    #1. jusText
    #text = justext.justext(html, justext.get_stoplist("English"))
    #text = '\n'.join([paragraph.text for paragraph in text if not paragraph.is_boilerplate])

    #2. boilerpy3
    #html = soup.body
    #text = extractor.get_content(soup.prettify())

    #3. Just extracting from 'text tags' like p
    #simple rules (does not work depending on website, like on artgallery.yale, anything without clear paragraphic style)
    #text = get_text_p_tags(soup)

    #4. NONE
    text = soup.body.get_text(' ').strip() # NOT GETTING FROM THE WHOLE SOUP, JUST BODY TO AVOID TITLES
    
    #POST PROCESSING
    text = apply_manual_rules(text)
    text = clean_text_line_by_line(text, ch_join = ' ', verb=verb, join=join)

    if not text:
        return 'No extractable text' if join else ['No extractable text']
    else:
        return text
i=578
print(i)
print(reference_html_df.url.iloc[i])
print(retrieve_text_from_html(reference_html_df.html.iloc[i], join=False))

578
https://bioguide.congress.gov/search/bio/G000240
0 Back Download { JSON } <XML> GLICKMAN, Daniel Robert 1944 – Overview Biography Congresses Bibliography Research Collections Collection of the U.S. House of Representatives, Gift of the U.S. Association of Former Members of Congress 9  US Congresses Served 95th – 103rd (1977 – 1995) House Years of Service 1977 – 1995 State / Territory Kansas Position Representative Party Democrat Overview 9  US Congresses Served 95th – 103rd (1977 – 1995) House Years of Service 1977 – 1995 State / Territory Kansas Position Representative Party Democrat Biography GLICKMAN, Daniel Robert,  a Representative from Kansas; born in Wichita, Sedgwick County, Kans., November 24, 1944; graduated from Southeast High School, Wichita, Kans., 1962; B.A., University of Michigan, Ann Arbor, Mich., 1966; J.D., George Washington University, Washington, D.C., 1969; lawyer, private practice; member of the Wichita, Kans., school board, 1973-1976; United States Securitie

In [7]:
reference_html_df['extracted_sentences'] = reference_html_df.html.progress_apply(retrieve_text_from_html, join=False, verb=False)

  0%|          | 0/676 [00:00<?, ?it/s]

# Re-separate sentences

As of now they are separated based on which tag element they were found. Let's unite it all and divide them based on punctuation

In [8]:
join_ch = ' '
reference_html_df['extracted_text'] = reference_html_df.extracted_sentences.apply(lambda x : join_ch.join(x))

In [9]:
from sentence_splitter import SentenceSplitter, split_text_into_sentences
splitter = SentenceSplitter(language='en')

from nltk.tokenize import sent_tokenize
import nltk

import pysbd
seg = pysbd.Segmenter(language="en", clean=False)

import spacy
nlp = spacy.load("en_core_web_lg")

In [10]:
text = reference_html_df.loc[0,'extracted_text']

# OPTION 1
# This gets some things wrong, such as Smt.=Shrimati ending a sentence, or any
# initials like P. N. Nampoothiri or Lt. Col.
#sents = sent_tokenize(text)

# OPTION 2
# Also breaks titles and initials like above, but additionally gets parenthesis wrong, like
# Amma Maharani [break](queen mother) [break] of Travancore.
#sents = seg.segment(text)

# OPTION 3
# Same as above plus new ones, like breaking contractions (like m. for married)
#sents = splitter.split(text)

# OPTION 4
# By far the best option, makes way less of the mistakes above, but not none. So let's adopt a strategy so ease this.
sents =  [s for s in nlp(text).sents]

# 
for i, sent in enumerate(sents):
    print(i, sent)

0 G. V. Raja.
1 From Wikipedia, the free encyclopedia.
2 This is an old revision of this page, as edited by Tom.
3 Reding (talk | contribs) at 17:06, 20 September 2019 (Category:CS1 errors: deprecated parameters: migrate 5/5 |dead-url= to |url-status=; minor cleanup; WP:GenFixes on).
4 The present address (URL) is a permanent link to this revision, which may differ significantly from the current revision.
5 Revision as of 17:06, 20 September 2019 by Tom.
6 Reding ( talk | contribs )
7 ( Category:CS1 errors: deprecated parameters: migrate 5/5 |dead-url= to |url-status=; minor cleanup; WP:GenFixes on) ( diff ) ← Previous revision | Latest revision ( diff ) | Newer revision → ( diff ) Jump to navigation.
8 Jump to search.
9 This article relies too much on references to primary sources.
10 Please improve this by adding secondary or tertiary sources.
11 ( December 2018 ) ( Learn how and when to remove this template message )
12 P. R. Godavarma Raja Karthika Nal Godavarma Raja Born 13 Octobe

We will store each singular sentence. We will *also* store the concatenation of every two adjacent sentences, in case they together store the right information, or were wrongly separated.


In [21]:
x = reference_html_df_i.loc[0,'extracted_text']
[s for s in nlp(x).sents]

[G. V. Raja.,
 From Wikipedia, the free encyclopedia.,
 This is an old revision of this page, as edited by Tom.,
 Reding (talk | contribs) at 17:06, 20 September 2019 (Category:CS1 errors: deprecated parameters: migrate 5/5 |dead-url= to |url-status=; minor cleanup; WP:GenFixes on).,
 The present address (URL) is a permanent link to this revision, which may differ significantly from the current revision.,
 Revision as of 17:06, 20 September 2019 by Tom.,
 Reding ( talk | contribs ),
 ( Category:CS1 errors: deprecated parameters: migrate 5/5 |dead-url= to |url-status=; minor cleanup; WP:GenFixes on) ( diff ) ← Previous revision | Latest revision ( diff ) | Newer revision → ( diff ) Jump to navigation.,
 Jump to search.,
 This article relies too much on references to primary sources.,
 Please improve this by adding secondary or tertiary sources.,
 ( December 2018 ) ( Learn how and when to remove this template message ),
 P. R. Godavarma Raja Karthika Nal Godavarma Raja Born 13 October 19

In [31]:
reference_html_df['nlp_sentences'] = reference_html_df.extracted_text.progress_apply(lambda x : [str(s) for s in nlp(x).sents])
reference_html_df['nlp_sentences']

  0%|          | 0/676 [00:00<?, ?it/s]

0      [G. V. Raja., From Wikipedia, the free encyclo...
1      [Sebastian Sabol., From Wikipedia, the free en...
2      [Average White Band., From Wikipedia, the free...
3      [Yutaka Higuchi (musician) From Wikipedia, the...
4      [Dolores Delirio., From Wikipedia, the free en...
                             ...                        
671    [Laila Majnu (1931) Director: J.J. Madan; Cast...
672    [Navjeevan (1935) Director: M. Bhavnani., Indi...
673    [Mantra Shakti (1935) Director: Satu Sen. Indi...
674    [Gul Sanobar (1934) Director: Homi Master; Wri...
675    [Jai Bhawani (1928) Director: K.P. Bhave., Ind...
Name: nlp_sentences, Length: 676, dtype: object

In [33]:
reference_html_df['nlp_sentences_slide_2'] = reference_html_df['nlp_sentences'].progress_apply(
    lambda x : [' '.join([a,b]) for a,b in list(zip(x,x[1:]+['']))]
)
reference_html_df['nlp_sentences_slide_2']

  0%|          | 0/676 [00:00<?, ?it/s]

0      [G. V. Raja. From Wikipedia, the free encyclop...
1      [Sebastian Sabol. From Wikipedia, the free enc...
2      [Average White Band. From Wikipedia, the free ...
3      [Yutaka Higuchi (musician) From Wikipedia, the...
4      [Dolores Delirio. From Wikipedia, the free enc...
                             ...                        
671    [Laila Majnu (1931) Director: J.J. Madan; Cast...
672    [Navjeevan (1935) Director: M. Bhavnani. India...
673    [Mantra Shakti (1935) Director: Satu Sen. Indi...
674    [Gul Sanobar (1934) Director: Homi Master; Wri...
675    [Jai Bhawani (1928) Director: K.P. Bhave. Indi...
Name: nlp_sentences_slide_2, Length: 676, dtype: object

In [48]:
assert type(reference_html_df.loc[0,'nlp_sentences']) == list
assert type(reference_html_df.loc[0,'nlp_sentences'][0]) == str
assert type(reference_html_df.loc[0,'nlp_sentences_slide_2']) == list
assert type(reference_html_df.loc[0,'nlp_sentences_slide_2'][0]) == str

In [39]:
reference_html_df.to_csv('reference_html_as_sentences_df.csv', index=None)

---

In [45]:
reference_html_df = pd.read_csv('reference_html_as_sentences_df.csv')

In [46]:
reference_html_df['nlp_sentences'] = reference_html_df['nlp_sentences'].apply(lambda x : ast.literal_eval(x))
reference_html_df['nlp_sentences_slide_2'] = reference_html_df['nlp_sentences_slide_2'].apply(lambda x : ast.literal_eval(x))

In [49]:
assert type(reference_html_df.loc[0,'nlp_sentences']) == list
assert type(reference_html_df.loc[0,'nlp_sentences'][0]) == str
assert type(reference_html_df.loc[0,'nlp_sentences_slide_2']) == list
assert type(reference_html_df.loc[0,'nlp_sentences_slide_2'][0]) == str